dada

In [ ]:
import cv2
import threading
from flask import Flask, jsonify
from flask_cors import CORS
from ultralytics import YOLO
import time

# Initialiserer Flask-applikationen
app = Flask(__name__)
CORS(app)  # Tillader cross-origin requests (CORS)

# Indlæser YOLO-modellen til at genkende terninger
model = YOLO(r"C:\Users\frode\Downloads\datasets\train21\weights\best.pt")
custom_classes = ["dice_1", "dice_2", "dice_3", "dice_4", "dice_5", "dice_6"]

# Initialiserer variabler til at holde styr på terningekast
latest_dice_roll = None  # Seneste terningekast
confirmed_dice_roll = None  # Bekræftet terningekast
dice_last_seen_time = time.time()  # Tidspunkt for seneste terningeregistrering
dice_removed = False  # Holder øje med, om terningen er blevet fjernet

# Åbner webcam til videooptagelse
cap = cv2.VideoCapture(0)

dawd

In [ ]:
def capture_video():
    """Funktion til at fange videostream og identificere terningekast."""
    global latest_dice_roll, confirmed_dice_roll, dice_last_seen_time, dice_removed
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Stopper hvis kameraet ikke længere sender billeder

        # Kører billedet gennem YOLO-modellen for at detektere objekter
        results = model(frame)[0]

        detected_dice = None
        for box in results.boxes:
            class_id = int(box.cls.item())  # Finder klassens ID
            if class_id < len(custom_classes):
                detected_dice = class_id + 1  # Tildeler værdi baseret på ID
                x1, y1, x2, y2 = map(int, box.xyxy[0])  # Henter koordinater for boks
                conf = box.conf.item()  # Henter sikkerhedsværdien (confidence)
                label = f"{custom_classes[class_id]} ({conf:.2f})"

                # Tegner boks omkring detekteret terning og viser label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        if detected_dice:
            if not dice_removed and detected_dice != latest_dice_roll:
                # Hvis terningen er blevet fjernet og en ny detekteres, opdateres værdien
                latest_dice_roll = detected_dice
                confirmed_dice_roll = detected_dice  # Bekræfter det nye kast
                dice_last_seen_time = time.time()  # Nulstiller timeren for terningen
            dice_removed = False  # Terningen er igen synlig
        else:
            # Hvis ingen terning er blevet set i 2 sekunder, markeres den som fjernet
            if not dice_removed and time.time() - dice_last_seen_time > 2:
                dice_removed = True  # Terningen er blevet fjernet
                latest_dice_roll = None  # Nulstiller sidste kast

        # Viser videofeed med detekterede terninger
        cv2.imshow("YOLOv8 Live Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  # Lukker vinduet, hvis 'q' trykkes

    cap.release()
    cv2.destroyAllWindows()  # Frigiver kameraet og lukker vinduerne

dawdw

In [ ]:
@app.route("/dice", methods=["GET"])
def get_dice():
    """API-endpoint til at hente det seneste terningekast."""
    global confirmed_dice_roll
    if confirmed_dice_roll is not None:
        roll = confirmed_dice_roll
        confirmed_dice_roll = None  # Nulstiller efter afsendelse
        return jsonify({"dice_roll": roll})
    return jsonify({"error": "No new dice roll"}), 404

def run_flask_app():
    """Starter Flask-serveren i en separat tråd."""
    app.run(host="127.0.0.1", port=5000, debug=False, use_reloader=False)

if __name__ == "__main__":
    # Starter Flask-serveren i en separat tråd
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.daemon = True
    flask_thread.start()

    time.sleep(2)  # Sikrer, at Flask er startet, før kameraet åbnes
    capture_video()  # Starter videooptagelse og detektering
